<a href="https://colab.research.google.com/github/hariszaf/metabolic_toy_model/blob/main/Antony2025/participantProjects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Finding Optimal Ecological Partners For Gut Colonization** (Julian Tap / Lu Chen)

\\

# **Impact of methanol on anaerobic digestion** (Ariane Bize)

\\

# **Understanding the communities in the anode and cathode of bioelectrosynthesis reactors** (Louise Rigaud / Wenbo Sui)

### **Basic Setup**

Setup **Gurobi**, **COBRApy**, **modelSEEDpy**, and **DNNGIOR**. See the [setting up your environment](https://colab.research.google.com/github/hariszaf/metabolic_toy_model/blob/main/Antony2025/preparingYourEnvironment.ipynb).

In [ ]:
import os
def create_gurobi_license():
    license_content = (
        "# Gurobi WLS license file\n"
        "# Your credentials are private and should not be shared or copied to public repositories.\n"
        "# Visit https://license.gurobi.com/manager/doc/overview for more information.\n"
        "WLSACCESSID=1fedf73b-9471-4da8-bdc7-2aaacf2e30f3\n"
        "WLSSECRET=3bc7d209-a4ec-4195-98be-4b254f181512\n"
        "LICENSEID=940603"
    )
    with open("/content/licenses/gurobi.lic", "w") as f:
        f.write(license_content)
    print("License file created at /content/licenses/gurobi.lic")



# Create directory for the license
os.makedirs("/content/licenses", exist_ok=True)

# Generate the license file
create_gurobi_license()

#add to path
os.environ['GRB_LICENSE_FILE'] = '/content/licenses/gurobi.lic'

License file created at /content/licenses/gurobi.lic


In [ ]:
# @title
!pip install gurobipy
!pip install cobra

!pip install pyrodigal
!pip install biopython

!pip install dnngior --no-deps

!git clone https://github.com/ModelSEED/ModelSEEDpy
!pip install ModelSEEDpy/.

!git clone https://github.com/hariszaf/metabolic_toy_model.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.8/141.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 586.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB

In [ ]:
# @title
import gurobipy
from gurobipy import Model
model = Model("test")
print("Gurobi is working!", "\U0001F600")

#install COBRApy
import cobra
from cobra.io import load_model
model = load_model("textbook")
solution = model.optimize()
print(f"flux balance analysis solution is {solution.objective_value}")
print("COBRApy is working", "\U0001F600")

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 940603
Academic license 940603 - for non-commercial use only - registered to da___@gmail.com
Gurobi is working! 😀
flux balance analysis solution is 0.8739215069684305
COBRApy is working 😀


### **Import all the packages we need**

In [ ]:
import numpy as np
import pandas as pd

import gzip
import shutil
import urllib

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import pyrodigal

import modelseedpy
from modelseedpy.core.msgenome import MSGenome

from modelseedpy.core.rast_client import RastClient
rast = RastClient()

from modelseedpy import MSBuilder

from dnngior.gapfill_class import *
from dnngior.NN_Predictor import NN


### **Gather all the data**

[Gut colonization data set](https://entrepot.recherche.data.gouv.fr/dataset.xhtml?persistentId=doi:10.57745/O4OJA2)

In [ ]:
genomes_gc = pd.read_csv("metabolic_toy_model/files/metasimfood_genomic_data.csv", sep = ';')
genomes_gc

,SPECIES_AND_STRAIN_NAME,TAXONOMY,ASSEMBLY_ACC_NUMBER,BIOPROJECT_ID,SAMPLE_ID,CHROMOSOME_ACC_NUMBER,GENOME_COVERAGE,GC_PERCENT,NB_REPLICONS,SIZE_CHROMOSOME_bp,SIZE_PLASMID_01_bp,SIZE_PLASMID_02_bp,SIZE_PLASMID_03_bp,SIZE_PLASMID_04_bp,SIZE_PLASMID_05_bp,SIZE_PLASMID_06_bp,SIZE_PLASMID_07_bp,SIZE_PLASMID_08_bp,SIZE_PLASMID_09_bp,SIZE_PLASMID_10_bp
0,Bacillus_pumilus_CIRM-BIA2784,k__Bacteria|p__Bacillota|c__Bacilli|o__Bacilla...,GCA_964063375,PRJEB74198,ERS18598044,OZ061327-OZ061328,330x,41.5,2,3753338,41278 (cluster_002),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Enterococcus_gilvus_CIRM-BIA2700,k__Bacteria|p__Bacillota|c__Bacilli|o__Lactoba...,GCA_964062525,PRJEB74198,ERS18598045,OZ061221-OZ061222,260x,42.0,2,2825867,889036 (cluster_002),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Lacticaseibacillus_paracasei_CIRM-BIA2373,k__Bacteria|p__Bacillota|c__Bacilli|o__Lactoba...,GCA_964065205,PRJEB74198,ERS18598046,OZ061570-OZ061573,440x,46.5,4,3088163,51021 (cluster_002),28481 (cluster_003),13402 (cluster_006),NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Lactiplantibacillus_pentosus_CNRZ1547,k__Bacteria|p__Bacillota|c__Bacilli|o__Lactoba...,GCA_964063425,PRJEB74198,ERS18598050,OZ061355-OZ061363,215x,46.0,9,3624812,67590 (cluster_002),54536 (cluster_003),58048 (cluster_005),33433 (cluster_006),33043 (cluster_009),22967 (cluster_007),14246 (cluster_008),3310 (cluster_017),NaN,NaN
4,Lactiplantibacillus_plantarum_ATCC14431,k__Bacteria|p__Bacillota|c__Bacilli|o__Lactoba...,GCA_964084915,PRJEB74198,ERS18598049,OZ064365-OZ064366,419x,44.5,2,3207663,9254 (cluster_004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Lactiplantibacillus_plantarum_CIRM-BIA2443,k__Bacteria|p__Bacillota|c__Bacilli|o__Lactoba...,GCA_964065245,PRJEB74198,ERS18598047,OZ061574-OZ061579,288x,44.5,6,3163810,40752 (cluster_002),33642 (cluster_003),24389 (cluster_004),10848 (cluster_005),8796 (cluster_006),NaN,NaN,NaN,NaN,NaN
6,Lactiplantibacillus_plantarum_CIRM-BIA2453,k__Bacteria|p__Bacillota|c__Bacilli|o__Lactoba...,GCA_964065445,PRJEB74198,ERS18598048,OZ061611-OZ061617,770x,44.5,7,3259765,52958 (cluster_007),39456 (cluster_009),36215 (cluster_013),26258 (cluster_017),16539 (cluster_024),8581 (cluster_027),NaN,NaN,NaN,NaN
7,Latilactobacillus_curvatus_CIRM-BIA2781,k__Bacteria|p__Bacillota|c__Bacilli|o__Lactoba...,GCA_964063355,PRJEB74198,ERS18598051,OZ061323-OZ061326,165x,42.0,4,2062891,43624 (cluster_002),22323 (cluster_003),12652 (cluster_004),NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Latilactobacillus_curvatus_J116,k__Bacteria|p__Bacillota|c__Bacilli|o__Lactoba...,GCA_964063385,PRJEB74198,ERS18598052,OZ061332-OZ061335,1000x,42.0,4,1928854,58746 (cluster_002),12649 (cluster_003),10908 (cluster_004),NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Latilactobacillus_sakei_CIRM-BIA1912,k__Bacteria|p__Bacillota|c__Bacilli|o__Lactoba...,GCA_964063755,PRJEB74198,ERS18598053,OZ061411-OZ061415,1200x,41.5,5,1907497,31814 (cluster_002),11126 (cluster_003),12662 (cluster_004),12470 (cluster_005),NaN,NaN,NaN,NaN,NaN,NaN


Methanol dataset

[see modelSEED biochemistry entry for methanol](https://modelseed.org/biochem/compounds/cpd00116)

In [ ]:
genomes_mt = pd.read_csv("metabolic_toy_model/files/contigs_coassembly-taxNames.tsv", sep = '\t')
genomes_mt.head()

,C,k127_911026,1797677,Bacteria; Firmicutes; Clostridiales; Clostridia; NA; NA; Clostridiales bacterium GWB2_37_7;
0,C,k127_121496,1036679,Archaea; Euryarchaeota; Methanosarcinales; Met...
1,C,k127_1154015,1036679,Archaea; Euryarchaeota; Methanosarcinales; Met...
2,C,k127_911050,1892845,Bacteria; Proteobacteria; Rhizobiales; Alphapr...
3,C,k127_546681,51616,Bacteria; Firmicutes; Clostridiales; Clostridi...
4,C,k127_125,53463,Bacteria; Proteobacteria; Rhodobacterales; Alp...


### **Write a function to download the genomes**

Three bins from the methanol project are in the folder `metabolic_toy_model/files/bins` (only to as examples)

In [ ]:
def download_fasta(url, gz_id, output):
  urllib.request.urlretrieve(url, gz_id)
  with gzip.open(gz_id, 'rb') as f_in:
      with open(output, 'wb') as f_out:
          shutil.copyfileobj(f_in, f_out)
  os.remove(gz_id)

### **Get the *Bifidobacterium* and the three first genomes from the list above in the a folder called food_bac_genomes**

In [ ]:
!mkdir food_bac_genomes

download_fasta("https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/010/425/GCF_000010425.1_ASM1042v1/GCF_000010425.1_ASM1042v1_protein.faa.gz",
               "GCF_000010425.1_ASM1042v1_protein.faa.gz",
               "food_bac_genomes/GCF_000010425.1_ASM1042v1_protein.faa")

download_fasta("https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/964/063/375/GCF_964063375.1_Bacillus_pumilus_CIRM-BIA2784/GCF_964063375.1_Bacillus_pumilus_CIRM-BIA2784_protein.faa.gz",
               "GCF_964063375.1_Bacillus_pumilus_CIRM-BIA2784_protein.faa.gz",
               "food_bac_genomes/GCF_964063375.1_Bacillus_pumilus_CIRM-BIA2784_protein.faa")

download_fasta("https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/964/062/525/GCF_964062525.1_Enterococcus_gilvus_CIRM-BIA2700/GCF_964062525.1_Enterococcus_gilvus_CIRM-BIA2700_protein.faa.gz",
               "GCF_964062525.1_Enterococcus_gilvus_CIRM-BIA2700_protein.faa.gz",
               "food_bac_genomes/GCF_964062525.1_Enterococcus_gilvus_CIRM-BIA2700_protein.faa")


download_fasta("https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/964/065/205/GCF_964065205.1_Lacticaseibacillus_paracasei_CIRM-BIA2373/GCF_964065205.1_Lacticaseibacillus_paracasei_CIRM-BIA2373_protein.faa.gz",
               "GCF_964065205.1_Lacticaseibacillus_paracasei_CIRM-BIA2373_protein.faa.gz",
               "food_bac_genomes/GCF_964065205.1_Lacticaseibacillus_paracasei_CIRM-BIA2373_protein.faa")




!ls food_bac_genomes -la

total 4468
drwxr-xr-x 2 root root    4096 Mar 13 08:25 .
drwxr-xr-x 1 root root    4096 Mar 13 08:25 ..
-rw-r--r-- 1 root root  736519 Mar 13 08:25 GCF_000010425.1_ASM1042v1_protein.faa
-rw-r--r-- 1 root root 1342664 Mar 13 08:25 GCF_964062525.1_Enterococcus_gilvus_CIRM-BIA2700_protein.faa
-rw-r--r-- 1 root root 1385571 Mar 13 08:25 GCF_964063375.1_Bacillus_pumilus_CIRM-BIA2784_protein.faa
-rw-r--r-- 1 root root 1095910 Mar 13 08:25 GCF_964065205.1_Lacticaseibacillus_paracasei_CIRM-BIA2373_protein.faa


#### **Write a function to reconstruct draft models**

We can make a small modification to the `reconstruct_draft_model` function we used yesterday to give the condition `is_dna_fasta` to know if we should annotate ORFs.

In [ ]:
def reconstruct_draft_model(model_id, input_fasta_file, output_model_sbml, is_dna_fasta = False):

  if is_dna_fasta:
    # Parse all records from the FASTA file
    records = list(SeqIO.parse(input_fasta_file, "fasta"))
    # Concatenate all sequence strings
    concatenated_sequence = "".join(str(record.seq) for record in records)
    concatenated_record = SeqRecord(Seq(concatenated_sequence),
                                id="concatenated",
                                description="Concatenated sequence of all records")
    dna_seq = str(concatenated_record.seq)  # Convert the sequence to a plain string
    gene_finder = pyrodigal.GeneFinder()
    gene_finder.train(dna_seq)
    genes = gene_finder.find_genes(dna_seq)

    with open(input_fasta_file + ".pep.fa", "w") as f:
      genes.write_translations(f, sequence_id="seqXYZ")

    input_fasta_file = input_fasta_file + ".pep.fa"

  genome = MSGenome.from_fasta(input_fasta_file, split = ' ')
  rast.annotate_genome(genome)

  base_model = MSBuilder.build_metabolic_model(model_id = model_id,
                                             genome   = genome,
                                             index    = "0",
                                             classic_biomass = True,
                                             gapfill_model   = False,
                                             gapfill_media   = None,
                                             annotate_with_rast = True,
                                             allow_all_non_grp_reactions = True
                                            )

  cobra.io.write_sbml_model(cobra_model = base_model, filename = output_model_sbml)

  return base_model

### **Build draft and gapfilled models**

##### **gut colonization**

we place them in the folder `food_bac_draft_models` and `food_bac_gapfilled_models`

In [ ]:
!mkdir food_bac_draft_models
!mkdir food_bac_gapfilled_models


genomes = os.listdir("food_bac_genomes")

for genome in genomes:
  model_id = genome.replace("_protein.faa", "")
  input_protein_fasta = "food_bac_genomes/" + genome
  output_model_sbml = "food_bac_draft_models/" + model_id + "_draft.sbml"
  draftModel = reconstruct_draft_model(model_id, input_protein_fasta, output_model_sbml)
  gapfilledModel = Gapfill(draftModel = output_model_sbml)
  cobra.io.write_sbml_model(gapfilledModel.gapfilledModel, filename = "food_bac_gapfilled_models/" + model_id + "_gapfilled.sbml")

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.


Gap-filling database =  ModelSEED


ERROR:cobra.io.sbml:No objective coefficients in model. Unclear what should be optimized


#reactions not found in NN-keys:  99 / 1007
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Flux through biomass reaction is 1.00000000
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  78616 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  39308 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  19654 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  9827 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  4913 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  2456 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently

ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.


...

#reactions not found in NN-keys:  65 / 725
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Flux through biomass reaction is 1.00000000
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  79158 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  39579 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  19789 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  9894 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  4947 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  2473 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:

ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.


Gap-filling database =  ModelSEED


ERROR:cobra.io.sbml:No objective coefficients in model. Unclear what should be optimized


#reactions not found in NN-keys:  98 / 1219
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Flux through biomass reaction is 1.00000000
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  78277 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  39138 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  19569 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  9784 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  4892 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  2446 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently

In [ ]:
!ls -la food_bac_draft_models
!ls -la food_bac_gapfilled_models

total 9576
drwxr-xr-x 2 root root    4096 Mar 13 08:31 .
drwxr-xr-x 1 root root    4096 Mar 13 08:25 ..
-rw-r--r-- 1 root root 1846499 Mar 13 08:29 GCF_000010425.1_ASM1042v1_draft.sbml
-rw-r--r-- 1 root root 2591373 Mar 13 08:25 GCF_964062525.1_Enterococcus_gilvus_CIRM-BIA2700_draft.sbml
-rw-r--r-- 1 root root 3085987 Mar 13 08:31 GCF_964063375.1_Bacillus_pumilus_CIRM-BIA2784_draft.sbml
-rw-r--r-- 1 root root 2267937 Mar 13 08:27 GCF_964065205.1_Lacticaseibacillus_paracasei_CIRM-BIA2373_draft.sbml
total 19268
drwxr-xr-x 2 root root    4096 Mar 13 08:32 .
drwxr-xr-x 1 root root    4096 Mar 13 08:25 ..
-rw-r--r-- 1 root root 4122578 Mar 13 08:30 GCF_000010425.1_ASM1042v1_gapfilled.sbml
-rw-r--r-- 1 root root 5141865 Mar 13 08:27 GCF_964062525.1_Enterococcus_gilvus_CIRM-BIA2700_gapfilled.sbml
-rw-r--r-- 1 root root 5802216 Mar 13 08:32 GCF_964063375.1_Bacillus_pumilus_CIRM-BIA2784_gapfilled.sbml
-rw-r--r-- 1 root root 4646354 Mar 13 08:28 GCF_964065205.1_Lacticaseibacillus_paracasei_CIRM-

##### **Methanol bins**

We place them in the folder `methanol_draft_models` and `methanol_gapfilled_models`. Set the parameter `is_dna_fasta` to `True`

In [ ]:
!mkdir methanol_draft_models
!mkdir methanol_gapfilled_models


genomes = os.listdir("metabolic_toy_model/files/bins")

for genome in genomes:
  model_id = genome.replace(".fa", "")
  input_fasta = "metabolic_toy_model/files/bins/" + genome
  output_model_sbml = "methanol_draft_models/" + model_id + "_draft.sbml"
  draftModel = reconstruct_draft_model(model_id, input_fasta, output_model_sbml, is_dna_fasta=True)
  gapfilledModel = Gapfill(draftModel = output_model_sbml)
  cobra.io.write_sbml_model(gapfilledModel.gapfilledModel, filename = "methanol_gapfilled_models/" + model_id + "_gapfilled.sbml")

mkdir: cannot create directory ‘methanol_draft_models’: File exists
mkdir: cannot create directory ‘methanol_gapfilled_models’: File exists


<ipython-input-22-c9db05c0042a>:5: ResourceWarning: unclosed file <_io.TextIOWrapper name='metabolic_toy_model/files/bins/bin.2.fa' mode='rt' encoding='UTF-8'>
  records = list(SeqIO.parse(input_fasta_file, "fasta"))
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.


Gap-filling database =  ModelSEED


ERROR:cobra.io.sbml:No objective coefficients in model. Unclear what should be optimized


#reactions not found in NN-keys:  22 / 110
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Flux through biomass reaction is 1.00000000
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  80216 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  40108 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  20054 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  10027 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  5013 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  2506 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently

...

#reactions not found in NN-keys:  25 / 282
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Flux through biomass reaction is 1.00000000
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  79907 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  39953 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  19976 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  9988 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  4994 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  2497 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:

<ipython-input-22-c9db05c0042a>:5: ResourceWarning: unclosed file <_io.TextIOWrapper name='metabolic_toy_model/files/bins/bin.1.fa' mode='rt' encoding='UTF-8'>
  records = list(SeqIO.parse(input_fasta_file, "fasta"))
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.


Gap-filling database =  ModelSEED


ERROR:cobra.io.sbml:No objective coefficients in model. Unclear what should be optimized


#reactions not found in NN-keys:  15 / 101
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Flux through biomass reaction is 1.00000000
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  80213 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  40106 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  20053 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  10026 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  5013 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  2506 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently

In [ ]:
!ls -la methanol_draft_models
!ls -la methanol_gapfilled_models

total 1272
drwxr-xr-x 2 root root   4096 Mar 13 08:49 .
drwxr-xr-x 1 root root   4096 Mar 13 08:32 ..
-rw-r--r-- 1 root root 269639 Mar 13 08:49 bin.1_draft.sbml
-rw-r--r-- 1 root root 283858 Mar 13 08:46 bin.2_draft.sbml
-rw-r--r-- 1 root root 734334 Mar 13 08:47 bin.3_draft.sbml
total 6400
drwxr-xr-x 2 root root    4096 Mar 13 08:51 .
drwxr-xr-x 1 root root    4096 Mar 13 08:32 ..
-rw-r--r-- 1 root root 1908407 Mar 13 08:51 bin.1_gapfilled.sbml
-rw-r--r-- 1 root root 1990524 Mar 13 08:47 bin.2_gapfilled.sbml
-rw-r--r-- 1 root root 2642104 Mar 13 08:49 bin.3_gapfilled.sbml


### **Now we have the models, what can we do with them?**

#### **Gut colonization**

#### **Environment ball**

Draw environments from a [Dirichlet distribution](https://en.wikipedia.org/wiki/Dirichlet_distribution)

In [ ]:
def generate_environment_ball(simul_n, transporters, outdir):
    '''
    simul_n: the number of random environments to generate.
    transporters: file containing list of trasporters (with modelSeed ids)
    outdir: directory to store results
    '''


    with open(transporters) as f:
        transporters=f.readline().strip().split('\t')
        names = f.readline().strip().split('\t')
    water_index=transporters.index('EX_cpd00001_e')

    oxygen_index=transporters.index('EX_cpd00007_e')


    random_environments=np.zeros((simul_n,len(transporters)))



    for i in range(simul_n):

        #Draw from a Dirichlet distribution
        a=np.random.dirichlet(np.ones(len(transporters)))

        #Set water to a constant value
        a[water_index]=1.0

        #Set a 50% chance of being anoxic
        if np.random.uniform()<0.5:
            a[oxygen_index]=0.0
        else:
            a[oxygen_index]=1.0

        #stretch the distribution
        a=a*1000
        random_environments[i]=a

    with open(os.path.join(outdir,   'env_ball_'+str(simul_n)+'.tsv'),'w') as f:

        f.write('\t'.join(transporters) + '\n')
        f.write('\t'.join(names) + '\n')


        for i in random_environments:

            for num in i:
                f.write('%.20f\t'%num)
            f.write('\n')

#Example of usage:
#main(1000, pathToFileWithListOfTransporters, pathToWriteOutput)

#### **Methanol**

![figure](files/figs/garza2020.png)